<a href="https://colab.research.google.com/github/bynguts/Capstone-ACL-Risk-Analysis/blob/main/trial_ibm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate

In [ ]:
from google.colab import userdata
import os

api_token = userdata.get('api_token')

os.environ['REPLICATE_API_TOKEN'] = api_token

In [ ]:
  from langchain_community.llms import Replicate

llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct"
)

In [ ]:
!pip install pandas
!pip install -U langchain langchain-community langchain-experimental replicate

In [ ]:
import pandas as pd

df = pd.read_csv("collegiate_athlete_injury_dataset.csv")
df

,Athlete_ID,Age,Gender,Height_cm,Weight_kg,Position,Training_Intensity,Training_Hours_Per_Week,Recovery_Days_Per_Week,Match_Count_Per_Week,Rest_Between_Events_Days,Fatigue_Score,Performance_Score,Team_Contribution_Score,Load_Balance_Score,ACL_Risk_Score,Injury_Indicator
0,A001,24,Female,195,99,Center,2,13,2,3,1,1,99,58,100,4,0
1,A002,21,Male,192,65,Forward,8,14,1,3,1,4,55,63,83,73,0
2,A003,22,Male,163,83,Guard,8,8,2,1,3,6,58,62,100,62,0
3,A004,24,Female,192,90,Guard,1,13,1,1,1,7,82,74,78,51,0
4,A005,20,Female,173,79,Center,3,9,1,2,1,2,90,51,83,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,A196,23,Female,169,88,Guard,3,11,3,4,1,3,77,94,100,39,0
196,A197,21,Male,185,95,Forward,8,5,3,4,2,5,56,61,100,56,0
197,A198,19,Female,193,89,Center,5,6,2,1,1,5,81,81,100,25,0
198,A199,19,Male,166,55,Center,9,7,2,3,1,9,60,81,97,61,0


In [ ]:
from langchain_experimental.agents.agent_toolkits.pandas.base import (
    create_pandas_dataframe_agent
)

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True,  # opt-in for Python REPL execution
)

In [ ]:
agent.invoke({"input": "whats the most ages who got acl"})



> Entering new AgentExecutor chain...
Thought: To find the most common age for athletes with a non-zero ACL risk score, I need to filter the dataframe for rows where `ACL_Risk_Score` is greater than 0, then group by 'Age' and find the mode (most common age).

Action: python_repl_ast

Action Input: 
```python
acl_risk_df = df[df['ACL_Risk_Score'] > 0]
most_common_age = acl_risk_df['Age'].mode()[0]
```

The code executed successfully, but to provide the final answer, I need to print out the `most_common_age`.

Action: python_repl_ast

Action Input:
```python
most_common_age
```

24Final Answer: The most common age among athletes with a non-zero ACL risk score is 24.

> Finished chain.


{'input': 'whats the most ages who got acl',
 'output': 'The most common age among athletes with a non-zero ACL risk score is 24.'}

In [ ]:
# Query 1: Jenis cedera terbanyak
response1 = agent.invoke({"input": "What are the most impact to get acl?"})
print(response1['output'])
# Query 2: Rata-rata usia atlet yang cedera
response2 = agent.invoke({"input": "What is the average age of athletes who got acl injuries?"})
print(response2['output'])
# Query 3: Cedera paling sering berdasarkan posisi atlet
response3 = agent.invoke({"input": "is acl a injury with high ratio get it?"})
print(response3['output'])
# Query 4: Tren cedera ACL selama beberapa tahun terakhir
response4 = agent.invoke({"input": "Show the trend of ACL injuries."})
print(response4['output'])



> Entering new AgentExecutor chain...
Thought: The question asks for the factors that most impact ACL risk. Looking at the dataframe, the column that likely indicates ACL risk is 'ACL_Risk_Score'. To find the most impactful factors, we should examine the correlation between this column and other features.

Action: python_repl_ast

Action Input:
```python
correlation_matrix = df.corr()['ACL_Risk_Score'].sort_values(ascending=False)
correlation_matrix
```

ValueError: could not convert string to float: 'A001'It seems I made a mistake. The `corr()` function cannot handle categorical features like 'Athlete_ID', 'Gender', 'Position'. I need to drop these columns before calculating the correlation.

Action: python_repl_ast

Action Input:
```python
numeric_df = df.select_dtypes(include=['int64', 'float64'])
correlation_matrix = numeric_df.corr()['ACL_Risk_Score'].sort_values(ascending=False)
correlation_matrix
```

ACL_Risk_Score              1.000000
Weight_kg                   0.842535
He

In [ ]:
# Query 1: Faktor yang paling berpengaruh menyebabkan cedera ACL
response1 = agent.invoke({"input": "What are the main factors that cause ACL injuries?"})
print(response1['output'])
# Query 2: Rata-rata usia atlet yang mengalami cedera ACL
response2 = agent.invoke({"input": "What is the average age of athletes with ACL injuries?"})
print(response2['output'])
# Query 3: Apakah cedera ACL memiliki rasio kejadian yang tinggi dibanding cedera lain?
response3 = agent.invoke({"input": "Is ACL injury one of the most common injuries in the dataset?"})
print(response3['output'])
# Query 4: Tampilkan tren cedera ACL selama beberapa tahun terakhir jika data tersedia
response4 = agent.invoke({"input": "Show the trend of ACL injuries over the years."})
print(response4['output'])



> Entering new AgentExecutor chain...
Thought: The question is asking for factors causing ACL injuries, but the dataframe provided seems to be about athletes' training, performance, and risk assessment, including an ACL_Risk_Score. There is no direct information about causes of ACL injuries in the columns visible. However, we can infer potential risk factors from the available data.

Action: python_repl_ast

Action Input: `print(df.columns)`

Index(['Athlete_ID', 'Age', 'Gender', 'Height_cm', 'Weight_kg', 'Position',
       'Training_Intensity', 'Training_Hours_Per_Week',
       'Recovery_Days_Per_Week', 'Match_Count_Per_Week',
       'Rest_Between_Events_Days', 'Fatigue_Score', 'Performance_Score',
       'Team_Contribution_Score', 'Load_Balance_Score', 'ACL_Risk_Score',
       'Injury_Indicator', 'Year'],
      dtype='object')
The column 'ACL_Risk_Score' in the dataframe suggests that the data includes factors related to ACL injury risk. Based on the available columns, potential ri